# Reinforcement Learning

Imagine an RL robot that needs to learn to take freshly painted cups from a painting machine, dry them, and place the dried cups on a shelf. We assign a +10 reward for storing dry cups on the shelf and a +5 reward for storing wet cups. The robot can attempt to dry any held cup, but there is a small chance (0.1) that the cup will break during the process, incurring a high cost (-20). If the robot decides to grab a new cup from the painting machine while already having a cup, it will drop and break the held cup (-20). The robot must now try to determine which policy to use for this task.



Let us use such a model to compare:
*   Policy Iteration (with the initial policy of a0 for all states)
*   Value Iteration (with the initial value of zero for all states)
*   First-Visit Monte Carlo (with the initial value of zero for all state-action pairs)
*   SARSA (with the initial value of zero for all state-action pairs)
*   Q-Learning

In [ ]:
import numpy as np

gamma = 1

a_count = 3
s_count = 4

pi = np.array([[1, 0, 0] for i in range(s_count)], dtype=np.float64)

P = np.array([[[0,1,0,0], [1,0,0,0], [1,0,0,0]],
     [[0,0,0,1], [0,0,.9,.1], [0,0,0,1]],
     [[0,0,0,1], [0,0,.9,.1], [0,0,0,1]],
     [[0,0,0,1], [0,0,0,1], [0,0,0,1]]], dtype=np.float64)

R = np.array([[[0,0,0,0], [0,0,0,0], [0,0,0,0]],
     [[0,0,0,-20], [0,0,0,-20], [0,0,0,5]],
     [[0,0,0,-20], [0,0,0,-20], [0,0,0,10]],
     [[0,0,0,0], [0,0,0,0], [0,0,0,0]]], dtype=np.float64)



In [ ]:
# policy iteration

H_eval = 10
H_pol = 5

def get_R(s, a):
  return np.sum(P[s][a] * R[s][a])

def get_V(s, a, V):
  return np.sum(P[s][a] * V)

def pol_eval(pi):
  V = np.array([0 for i in range(s_count)], dtype=np.float64)
  for k in range(H_eval):
    V_k = V.copy()
    for s in range(s_count):
      sum = 0.0
      for i, pa in enumerate(pi[s]):
        sum += pa * (get_R(s,i) + gamma * get_V(s,i,V))
      V_k[s] = sum
    V = V_k
  return V

def pol_equals(pi1, pi2):
  for i in range(s_count):
    for j in range(a_count):
      if abs(pi1[i][j] - pi2[i][j]) > 0.0001:
        return False
  return True

def pol_imp(pi):
  V = pol_eval(pi)
  for k in range(H_pol):
    print("Value: " + str(V))
    print("Policy: " + str(pi))
    pi_k = pi.copy()
    for s in range(s_count):
      max_ = np.NINF
      max_i = -1
      for i in range(a_count):
        rew = get_R(s,i) + gamma * get_V(s,i,V)
        if rew > max_:
          max_ = rew
          max_i = i
      pi_k[s] = np.array([0, 0, 0], dtype=np.float64)
      pi_k[s][max_i] = 1.0
    V = pol_eval(pi_k)
    if pol_equals(pi, pi_k):
      print("Value: " + str(V))
      break
    pi = pi_k

pol_imp(pi)

Value: [-20. -20. -20.   0.]
Policy: [[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]
Value: [ 5.  5. 10.  0.]
Policy: [[1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]]
Value: [ 7.  7. 10.  0.]
Policy: [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]]
Value: [ 7.  7. 10.  0.]


In [ ]:
# value iteration

V = np.array([0 for i in range(4)], dtype=np.float64)

H = 5 # horizon, hyperparameter

for k in range(H):
  V_k = V.copy()
  for s in range(s_count):
    max_ = np.NINF
    for i in range(a_count):
      rew = get_R(s,i) + gamma * get_V(s,i,V)
      if rew > max_:
        max_ = rew
    V_k[s] = max_
  V = V_k

  print(V)

[ 0.  5. 10.  0.]
[ 5.  7. 10.  0.]
[ 7.  7. 10.  0.]
[ 7.  7. 10.  0.]
[ 7.  7. 10.  0.]


In [ ]:
# first-visit monte carlo
import random

eps = .2

H = 10
epochs = 40000

Q = np.array([[0,0,0], [0,0,0], [0,0,0], [0,0,0]], dtype=np.float64)
Q_c = [[0,0,0], [0,0,0], [0,0,0], [0,0,0]]

def get_act(s, Q, eps):
  rand = random.uniform(0, 1)
  if rand > eps:
    return np.argmax(Q[s])
  else:
    return random.randrange(0,a_count)

def get_next_s(s, a):
  p = P[s][a]
  rand = random.uniform(0, 1)
  sum = 0
  for i in range(s_count):
    sum += p[i]
    if sum >= rand:
      return i, R[s][a][i]

def update(stack):
  Q_ct = np.zeros((4,3), dtype=np.float64)
  Qt = np.zeros((4,3), dtype=np.float64)
  for i, obs in enumerate(stack):
    s, a, rew = obs
    if Q_ct[s][a] == 0:
      Q_ct[s][a] = 1
      sum = 0
      c = 0
      for j in range(i, len(stack)):
        st, at, rewt = stack[j]
        sum += rewt
        c += 1
      Qt[s][a] = sum / c

  for s in range(s_count):
    for a in range(a_count):
      if Q_ct[s][a] > 0:
        c = Q_c[s][a]
        Q_c[s][a] = c + 1
        Q[s][a] = (c * Q[s][a] + Qt[s][a]) / (c + 1)

for i in range(epochs):
  stack = []
  s = 0
  for j in range(H):
    a = get_act(s, Q, eps)
    s_next, rew = get_next_s(s, a)
    stack.append([s,a,rew])
    s = s_next
  update(stack)
  print(Q)


Görüntülenen çıkış son 5000 satıra kısaltıldı.
[[ 0.33846748  0.32364047  0.31699376]
 [-2.27119443  0.57998018  0.56709818]
 [-2.58120748  0.6724697   1.29259608]
 [ 0.          0.          0.        ]]
[[ 0.33848458  0.32364047  0.31699376]
 [-2.27119443  0.58009964  0.56709818]
 [-2.58120748  0.6724697   1.29258463]
 [ 0.          0.          0.        ]]
[[ 0.33850167  0.32364047  0.31699376]
 [-2.27119443  0.58021905  0.56709818]
 [-2.58120748  0.6724697   1.29257319]
 [ 0.          0.          0.        ]]
[[ 0.33844124  0.32364047  0.31699376]
 [-2.27119443  0.57958901  0.56709818]
 [-2.58091849  0.6724697   1.29257319]
 [ 0.          0.          0.        ]]
[[ 0.33845834  0.32364047  0.31699376]
 [-2.27119443  0.57970848  0.56709818]
 [-2.58091849  0.6724697   1.29256176]
 [ 0.          0.          0.        ]]
[[ 0.33847543  0.32364047  0.31699376]
 [-2.27119443  0.57982789  0.56709818]
 [-2.58091849  0.6724697   1.29255034]
 [ 0.          0.          0.        ]]
[[ 0.338492

In [ ]:
# sarsa

import random

eps = .2
alpha = .2

H = 10
epochs = 10000

Q = np.array([[0,0,0], [0,0,0], [0,0,0], [0,0,0]], dtype=np.float64)
Q_c = [[0,0,0], [0,0,0], [0,0,0], [0,0,0]]

def get_act(s, Q, eps):
  rand = random.uniform(0, 1)
  if rand > eps:
    return np.argmax(Q[s])
  else:
    return random.randrange(0,a_count)

def get_next_s(s, a):
  p = P[s][a]
  rand = random.uniform(0, 1)
  sum = 0
  for i in range(s_count):
    sum += p[i]
    if sum >= rand:
      return i, R[s][a][i]

def update(stack):
  for obs in stack:
    s, a, rew = obs
    c = Q_c[s][a]
    Q_c[s][a] = c + 1
    Q[s][a] = (c * Q[s][a] + rew) / (c + 1)

for i in range(epochs):
  s = 0
  for j in range(H):
    a = get_act(s, Q, eps)
    s_next, rew = get_next_s(s, a)
    a_next = get_act(s_next, Q, eps)
    Q[s][a] = Q[s][a] + alpha*(rew + gamma*Q[s_next][a_next] - Q[s][a])
    s = s_next
  print(Q)


Görüntülenen çıkış son 5000 satıra kısaltıldı.
[[  1.71128427   1.22245729   1.91483457]
 [-20.           4.04131249   5.        ]
 [-19.99999486   7.21664467  10.        ]
 [  0.           0.           0.        ]]
[[  2.36902742   1.36093274   1.91483457]
 [-20.           5.23304999   5.        ]
 [-19.99999486   7.21664467  10.        ]
 [  0.           0.           0.        ]]
[[ -2.10477806   1.36093274   1.91483457]
 [-20.           5.23304999   5.        ]
 [-19.99999486   7.21664467  10.        ]
 [  0.           0.           0.        ]]
[[ -2.10477806   0.75641487   1.11091204]
 [-20.           5.23304999   5.        ]
 [-19.99999486   7.21664467  10.        ]
 [  0.           0.           0.        ]]
[[ -0.63721245   0.75641487   1.04001261]
 [-20.           0.18644102   5.        ]
 [-19.99999486   7.21664467  10.        ]
 [  0.           0.           0.        ]]
[[  0.49023004   0.81313442   1.04001261]
 [-20.           0.18644102   5.        ]
 [-19.99999486   7.21664

In [ ]:
# Q learning

import random

eps = .2
alpha = .2
gamma = .9

H = 10
epochs = 10000

Q = np.array([[0,0,0], [0,0,0], [0,0,0], [0,0,0]], dtype=np.float64)
Q_c = [[0,0,0], [0,0,0], [0,0,0], [0,0,0]]

def get_act(s, Q, eps):
  rand = random.uniform(0, 1)
  if rand > eps:
    return np.argmax(Q[s])
  else:
    return random.randrange(0,a_count)

def get_next_s(s, a):
  p = P[s][a]
  rand = random.uniform(0, 1)
  sum = 0
  for i in range(s_count):
    sum += p[i]
    if sum >= rand:
      return i, R[s][a][i]

def update(stack):
  for obs in stack:
    s, a, rew = obs
    c = Q_c[s][a]
    Q_c[s][a] = c + 1
    Q[s][a] = (c * Q[s][a] + rew) / (c + 1)

for i in range(epochs):
  s = 0
  for j in range(H):
    a = get_act(s, Q, eps)
    s_next, rew = get_next_s(s, a)
    a_next = np.argmax(Q[s_next])
    Q[s][a] = Q[s][a] + alpha*(rew + gamma*Q[s_next][a_next] - Q[s][a])
    s = s_next
  print(Q)


Görüntülenen çıkış son 5000 satıra kısaltıldı.
[[  4.5          4.0549512    4.07545861]
 [-20.           2.37475558   5.        ]
 [-20.           2.70049862  10.        ]
 [  0.           0.           0.        ]]
[[  4.5          4.0549512    4.07545861]
 [-20.           2.37475558   5.        ]
 [-20.           2.70049862  10.        ]
 [  0.           0.           0.        ]]
[[  4.5          4.0549512    4.07545861]
 [-20.           2.37475558   5.        ]
 [-20.           2.70049862  10.        ]
 [  0.           0.           0.        ]]
[[  4.5          4.0549512    4.07545861]
 [-20.           2.37475558   5.        ]
 [-20.           2.70049862  10.        ]
 [  0.           0.           0.        ]]
[[  4.5          4.0549512    4.07545861]
 [-20.           2.37475558   5.        ]
 [-20.           2.70049862  10.        ]
 [  0.           0.           0.        ]]
[[  4.5          4.0549512    4.07545861]
 [-20.           2.37475558   5.        ]
 [-20.           2.70049

# Comparison

To compare the results, we see that they are quite similar for each algorithm.

One big difference is that, unlike MC and SARSA, Q-learning does not recommend a0 at S0 (grabbing a cup when empty handed). However, it should be noted that Q-learning also attributes a0 to S0, when γ is 0.9 instead of 1.

Also, we see that the iteration-based approaches recommend a1 at S1, instead of a2. This is not surprising, as they would overlook the dangers of drying a wet cup.

To compare the convergence rates, policy and value iteration converges pretty quickly, whereas Monte Carlo, SARSA and Q-learning takes a long time to converge. To a point where my IDE had to limit the output shown.


This was expected, as these three methods are model-free and work based on experience.

Since we do not have access to certain probabilities, we should use a model-free algorithm to find the optimal policy. Those won’t need the transition matrix and will work merely from experience.

Thus, we should use one of the following:
*   First-Visit Monte Carlo
*   SARSA
*   Q-Learning

Specifically, we may go for SARSA as it can converge faster than Q-learning. Also, TD may converge quicker than MC.

# What about ϵ?

ϵ incorporates randomness in our algorithm. In other words, it helps the algorithm explore as opposed to exploit. If we increase ϵ, the algorithm will explore more, and exploit less. It will take random actions more frequently.